In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import pandas as pd
from tqdm import tqdm
import string
tqdm.pandas()
#import numpy as np

df = pd.read_csv("MyData.csv")

In [4]:
train = df.loc[0:50,]
#print(train['text'])
test = df.loc[51:len(df),]
#test

In [5]:
print("Train shape : ",train.shape)
print("Test shape : ",test.shape)

Train shape :  (51, 110)
Test shape :  (26, 110)


In [6]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence.split():
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [7]:
vocab = build_vocab(train['text'])
print({k: vocab[k] for k in list(vocab)[:5]})

100%|██████████| 51/51 [00:00<00:00, 3407.12it/s]


{'20%': 3, 'bonus': 2, 'for': 267, 'registered': 3, 'developers': 7}


In [8]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import word2vec
from gensim.models import KeyedVectors

C:\Users\Public\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
EMBEDDING_FILE = 'glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding = 'utf-8'))

In [10]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [11]:
oov = check_coverage(vocab,embeddings_index)

100%|██████████| 8423/8423 [00:00<00:00, 351899.74it/s]


Found embeddings for 69.23% of vocab
Found embeddings for  88.29% of all text


In [12]:
#for punct in ';{}()[].=$<>#,':
#    print(punct in embeddings_index)

In [13]:
to_remove = ['a','to','of','and']
def remove_words(x):
    x = str(x)
    y = []
    for w in x.split():
        if w not in to_remove:
            y.append(w)    
    return " ".join(y)

def remove_punc(x):

    x = str(x)
    for punct in "/-'"  + '\n' + '\r':
        x = x.replace(punct, ' ')
    for punct in ';{}()[].=$<>#,':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!,"%\'*+-/:&@\\^_`|~' + '“”’':
        x = x.replace(punct, '')
    
    return x.lower()

def clean_text(x):
    x = remove_punc(x)
    x = remove_words(x)
    return x

In [14]:
train["text"] = train["text"].progress_apply(lambda x: clean_text(x))
test["text"] = test["text"].progress_apply(lambda x: clean_text(x))
vocab = build_vocab(train["text"])

100%|██████████| 51/51 [00:00<00:00, 700.23it/s]
C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
100%|██████████| 26/26 [00:00<00:00, 1737.85it/s]
C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
100%|██████████| 51/51 [00:00<00:00, 3196.02it/s]


In [15]:
oov = check_coverage(vocab,embeddings_index)

100%|██████████| 5731/5731 [00:00<00:00, 338033.42it/s]


Found embeddings for 95.39% of vocab
Found embeddings for  98.77% of all text


In [16]:
train.to_csv('cleaned_train.csv',index=False)
test.to_csv('cleaned_test.csv',index=False)

In [17]:
test

,id,url,X,Title,Subtitle,Image,Author,Publication,Year,Month,...,Tag_trump,Tag_ux,Tag_venture.capital,Tag_web.design,Tag_web.development,Tag_women,Tag_wordpress,Tag_work,Tag_writing,text
51,52,https://medium.com/@uctoday/businesses-are-inv...,301705,Businesses Are Investing in Tech to Improve CX...,0,1,UC Today,0,2018,6,...,0,0,0,0,0,0,0,0,0,new mitel survey 5 000 adults from the uk fran...
52,53,https://medium.com/@vepo/techniques-and-civili...,304060,Techniques and CivilizationAgents of Mechaniza...,Agents of Mechanization,0,Victor Os rio,0,2017,11,...,0,0,0,0,0,0,0,0,0,this is summary from the chapter agents mechan...
53,54,https://medium.com/@VerhaalNg/how-social-media...,1117,How Social Media Can Help Your Brand,Social media is an effective marketing tool th...,1,Verhaal NG,0,2017,8,...,0,0,0,0,0,0,0,0,0,social media is an effective marketing tool th...
54,55,https://medium.com/@victoriainthewoods/addicte...,304679,Addicted,To the sound of the oceanSurrealism of being.C...,0,Victoria Woods,0,2017,9,...,0,0,0,0,0,0,0,0,1,the sound the oceansurrealism being . centered...
55,56,https://medium.com/@whatjosaid/feminist-sex-th...,308620,Feminist sex & the fleshy question of desire,0,0,Jo Murphy,0,2017,9,...,0,0,0,0,0,0,0,0,0,yes thats me in the red swimsuit aged 16 full ...
56,57,https://medium.com/@WhiteSourceSoft/the-equifa...,6214,"The Equifax Hack: 6 Months Later, What Did We ...",0,1,WhiteSource,0,2018,4,...,0,0,0,0,0,0,0,0,0,when news broke that credit rating agency equi...
57,58,https://medium.com/@wreakhavoc1/she-needs-to-a...,4027,0,0,0,askyourselfquestions,0,2017,10,...,0,0,0,0,0,0,0,0,0,she needs accept her realityshe needs watch wh...
58,59,https://medium.com/@ZoomZoom/last-blog-6ad17f2...,2629,Last Blog??,This is the last blog post I am required to do.,1,Yevgeniy Semenyuk,0,2017,12,...,0,0,0,0,0,0,0,0,0,this is the last blog post i am required do . ...
59,60,https://medium.com/aurora-dao/how-to-create-an...,319750,How to create and connect your private key/key...,private key,0,Philip Wearn,Aurora DAO,2017,11,...,0,0,0,0,0,0,0,0,0,heres how connect your wallet with private key...
60,61,https://medium.com/employers-of-late-capitalis...,362,What can you find here and why I chose to laug...,0,1,employers of late capitalism,employers of late capitalism,2017,8,...,0,0,0,0,0,0,0,0,0,tl ; dr laughing at the ridiculous current sta...


In [18]:
train

,id,url,X,Title,Subtitle,Image,Author,Publication,Year,Month,...,Tag_trump,Tag_ux,Tag_venture.capital,Tag_web.design,Tag_web.development,Tag_women,Tag_wordpress,Tag_work,Tag_writing,text
0,1,https://blog.fundrequest.io/a-token-sale-by-an...,8795,A Token Sale by and for Developers,20% bonus for registered developers who take p...,1,Karel Striegel,FundRequest.io,2017,11,...,0,0,0,0,0,0,0,0,0,20 bonus for registered developers who take pa...
1,2,https://fityourself.club/when-effective-we-gen...,20214,"When Effective, we generate and surround ourse...","When we focus too much on the results, we lose...",0,"Jen Mak, A Space For",Fit Yourself Club,2018,2,...,0,0,0,0,0,0,0,0,0,by attending the right problems remaining calm...
2,3,https://levelup.gitconnected.com/a-one-week-st...,35885,"A one week stand with React NativeOr, how deep...",Our teams experience with trying,1,James Adams,gitconnected Become a Better Developer,2018,5,...,0,0,0,0,1,0,0,0,0,our team is huddled around black wall littered...
3,4,https://medium.com/@ACWTCOACHING/glittering-pr...,11934,"Glittering Prizechase values, not the carrot",0,1,Kevin M. Crawford,0,2018,2,...,0,0,0,0,0,0,0,0,0,in recent linkedin interview diane von fursten...
4,5,https://medium.com/@alex.brigham/insecure-old-...,5273,Insecure? Old Spice and Dove can help!,"If you own a television and have cable, theres...",0,Alex Brigham,0,2017,11,...,0,0,0,0,0,0,0,0,0,if you own television have cable theres good c...
5,6,https://medium.com/@ashleecraft/2018-week-2-ac...,12110,2018 Week 2: Accountability Update,Im doing a weekly set of posts where I outline...,1,Ashlee Craft,0,2018,1,...,0,0,0,0,0,0,0,0,0,im doing weekly set posts where i outline what...
6,7,https://medium.com/@AuthorSteger/how-to-accept...,7073,How To Accept Criticism,And Handle It With Some Grace.,1,Johannus M. Steger,0,2017,9,...,0,0,0,0,0,0,0,0,1,part who we all are is deeply ground within ou...
7,8,https://medium.com/@cancergeek/a-design-a-flaw...,127808,"A Design, A flaw, & A Sharpie",0,0,CancerGeek,0,2017,8,...,0,0,0,0,0,0,0,0,0,by cancergeek we recently launched the # gfhcp...
8,9,https://medium.com/@cebsilver/newsletter-of-in...,129750,Newsletter of Industry 2/6/2018,0,1,Curtis Silver,0,2018,2,...,0,0,0,0,0,0,0,0,0,this eight foot long naugahyde couch comes dir...
9,10,https://medium.com/@ceotter/death-by-chips-and...,20770,Death By Chips and Salsa,Food has a tendency of bringing people togethe...,1,Caroline Otter,0,2018,3,...,0,0,0,0,0,0,0,0,0,food has tendency bringing people together tha...
